In [1]:
import pandas as pd
from os import listdir, path, makedirs
from datetime import datetime
from collections import defaultdict
from xlrd import XLRDError

In [2]:
# Folder path
%cd /mnt/data/vieri/projects/SAMPLE/

/mnt/data/vieri/projects/SAMPLE


In [3]:
system_names = ["Cantore", "Emi", "Soleto 1", "Soleto 2","Galatina", "Verone"]
system_name = system_names[1] # OK: 0, 1, 2, 3, 4, 5

In [4]:
# TASK: Retrive file paths (inverter data + irradiance data)
folder_path = path.join("data", system_name.upper(), system_name.upper())
irr_file_name = "Solarimetro"

# Retrive file paths (for inverter data & irradiance data)
inv_files = dict()
irr_files = []
for item_name in listdir(folder_path):
    item_path = path.join(folder_path, item_name)
    if path.isdir(item_path):
        if "INV" in item_name.upper():
            inv_files[item_name] = [file for file in listdir(item_path) if file.endswith(('.xlsx', '.xls'))]
        elif item_name.upper() == irr_file_name.upper():
            irr_files = [file for file in listdir(item_path) if file.endswith(('.xlsx', '.xls'))] 

# Create and sort alphabetically a list of the inverter names 
inv_names = list(inv_files.keys())
inv_names.sort()

# In case irradiance data is unavailable --> Use data from a neighbour system
# This approach is valid only for Soletto 2 & Galatina (since they are close to Soletto 1)
if not irr_files:
    neighbour_system = "SOLETO 1"
    irr_data_neighbour_system = path.join("data", neighbour_system, neighbour_system, "Solarimetro")
    irr_files = [file for file in listdir(irr_data_neighbour_system) if file.endswith(('.xlsx', '.xls'))] 
    print("WARNING: Irradiance data seems to not be availabe for this PV system ({0}).\nIrradiance data from '{1}' will be used"
          .format(neighbour_system, neighbour_system))
            
# Remove the useless file containing the daily irradiance values
file_to_delate = [file_name for file_name in irr_files if len(file_name) > len(system_name) + 17]
if file_to_delate:
    irr_files.remove(file_to_delate[0])
    print("\nWARNING: The file containing the averaged daily irradiance values ({0}) has been skipped.\n".format(file_to_delate[0]))

# Order file paths according to the data included in the file name
mapping_func = lambda date: datetime.strptime(date[-11:-4],'%Y-%m')
irr_files.sort(key = mapping_func)
[inv_files[inv_name].sort(key = mapping_func) for inv_name in inv_names]   

# Display number of files retrieved
print(len(inv_files[inv_names[0]]), "month files have been loaded for each of the", len(inv_names), "inverter data (", ", ".join(inv_names), ")")
print(len(irr_files), "files of montly irradiance data have been loaded")
period_covered = (inv_files[inv_names[0]][0][-11:-4], inv_files[inv_names[0]][-1][-11:-4])
print("\nINVERTER PERIOD COVERED: From {0} to {1}".format(period_covered[0], period_covered[1]))
    
if len(inv_files[inv_names[0]]) != len(irr_files):
    irr_period_covered = (irr_files[0][-11:-4], irr_files[-1][-11:-4])
    print("IRRADIANCE PERIOD COVERED: From {0} to {1}".format(irr_period_covered[0], irr_period_covered[1]))



36 month files have been loaded for each of the 4 inverter data ( Inv.1, Inv.2, Inv.3, Inv.4 )
36 files of montly irradiance data have been loaded

INVERTER PERIOD COVERED: From 2018-07 to 2021-06


In [5]:
# TASK: Read and import data as dataframes 
# A) Read monthly inverter data
inv_months = defaultdict(list)
for inv_name in inv_names:
    print("\nReading the {0} data (FROM '{1}' TO '{2}')... ".format(inv_name,period_covered[0], period_covered[1]))
    files_to_read = inv_files[inv_name]
    skipped_files = []
    for file in files_to_read:
        try:
            inv_months[inv_name].append(pd.read_excel(path.join(folder_path, inv_name, file)))
        except (ValueError, XLRDError):
            skipped_files.append(file)
            
    if skipped_files:
        print("WARNING!:", len(skipped_files), "files (", round((len(skipped_files)/len(files_to_read))*100, 1), "%) have been skipped since it is empty")
            
print("OK: Inverter montly data ({2} files:{0} inv. data (x) {1} months) has been loaded\n"
      .format(len(inv_names), len(inv_months[inv_names[0]]),len(inv_names)* len(inv_months[inv_names[0]])))

# B) Read monthly irradiance data
print("Reading irradiance data...")
irr_months = []
for irr_file in irr_files:
    print("Reading data from " + irr_file[-11:-4] + " ...")
    try:
        irr_months.append(pd.read_excel(path.join(folder_path, irr_file_name, irr_file)))
    except FileNotFoundError: # Use the variable with the paths of the neighbour system 
        irr_months.append(pd.read_excel(path.join(irr_data_neighbour_system, irr_file)))
        
print("OK: Irradiance montly data ({0}) has been loaded\n".format(len(irr_months)))


Reading the Inv.1 data (FROM '2018-07' TO '2021-06')... 

Reading the Inv.2 data (FROM '2018-07' TO '2021-06')... 

Reading the Inv.3 data (FROM '2018-07' TO '2021-06')... 

Reading the Inv.4 data (FROM '2018-07' TO '2021-06')... 
OK: Inverter montly data (144 files:4 inv. data (x) 36 months) has been loaded

Reading irradiance data...
Reading data from 2018-07 ...
Reading data from 2018-08 ...
Reading data from 2018-09 ...
Reading data from 2018-10 ...
Reading data from 2018-11 ...
Reading data from 2018-12 ...
Reading data from 2019-01 ...
Reading data from 2019-02 ...
Reading data from 2019-03 ...
Reading data from 2019-04 ...
Reading data from 2019-05 ...
Reading data from 2019-06 ...
Reading data from 2019-07 ...
Reading data from 2019-08 ...
Reading data from 2019-09 ...
Reading data from 2019-10 ...
Reading data from 2019-11 ...
Reading data from 2019-12 ...
Reading data from 2020-01 ...
Reading data from 2020-02 ...
Reading data from 2020-03 ...
Reading data from 2020-04 ...
R

In [6]:
# TASK: Merge all of the montly data into a single dataframe 
inv_dfs = dict()
for inv_name in inv_names:
    # A) Merge montly inverter data 
    inv_dfs[inv_name] = pd.concat(inv_months[inv_name], ignore_index=True) #, sort=False
    
    # Cast "Allarme" as string instead of an integer
    inv_dfs[inv_name]["Allarme"] = inv_dfs[inv_name]["Allarme"].astype("string")
display(inv_dfs[inv_names[0]].info())

# B) Merge monthly irradiance data
irr_dfs = pd.concat(irr_months, ignore_index=True)
display(irr_dfs.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149873 entries, 0 to 149872
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Data             149873 non-null  datetime64[ns]
 1   Iac R (A)        149873 non-null  int64         
 2   Iac S (A)        149873 non-null  int64         
 3   Iac T (A)        149873 non-null  int64         
 4   Vac R (V)        149873 non-null  int64         
 5   Vac S (V)        149873 non-null  int64         
 6   Vac T (V)        149873 non-null  int64         
 7   Pac R (kW)       149873 non-null  int64         
 8   Pac S (kW)       0 non-null       float64       
 9   Pac T (kW)       0 non-null       float64       
 10  E. totale (kWh)  149873 non-null  float64       
 11  Cc 1 (A)         149873 non-null  int64         
 12  Vcc 1 (V)        149873 non-null  int64         
 13  Allarme          149873 non-null  string        
 14  Stato            0 n

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238822 entries, 0 to 238821
Data columns (total 2 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   data               238822 non-null  datetime64[ns]
 1   irr. medio 1 W/mq  238796 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.6 MB


None

In [7]:
# TASK: Merge the two dataframes (inverter data and irradiance data)
full_inv_data = dict()
perfect_match = dict()
for inv_name in inv_names:
    print("INVERTER DATA: ", inv_name)
    
    # Perform a Left outer join
    full_inv_data[inv_name] = inv_dfs[inv_name].merge(irr_dfs, how="left", left_on="Data", right_on="data")
    
    # Subtle transformations on the dataframe
    full_inv_data[inv_name].drop(columns = "data", inplace = True)
    full_inv_data[inv_name].sort_values(by=['Data'], inplace=True)
    full_inv_data[inv_name].rename(inplace=True, columns = {"Data" : "Date/Time",
                                                            "°C" : "Inverter temp. (°C)",
                                                            "irr. medio 1 W/mq" : "Irr. medio (W/mq)"})
    
    # Register and visualize a warning 
    # WHAT: Check wheather there is a mismatch between inverter data and irradiance data
    # SINCE: Emperically, many mismatches (i.e., desynchronizations) have been found 
    # MOTIVATION: In case of mismatches save the raw irradiance values as well as the merged dataframe
    if len(full_inv_data[inv_name]) != len(irr_dfs):
        perfect_match[inv_name] = False
        print("WARNING: Some mismatches (i.e., desynchronized obs.) [{0}] have been discovered during the merge process."
              .format(abs(len(full_inv_data[inv_name]) - len(irr_dfs))))
        print("|A left outer join has been performed (i.e., keeping all observations of the {0} data)".format(inv_name))
        print("|The merged dataframe as well as the two dataframes will be saved\n")
    else:
        perfect_match[inv_name] = True
        print("Perfect match between {0} and the irradiance values".format(inv_name))
        print("|A inner join has been performed\n")
        print("|Only the merged dataframe will be saved\n")

INVERTER DATA:  Inv.1
|A left outer join has been performed (i.e., keeping all observations of the Inv.1 data)
|The merged dataframe as well as the two dataframes will be saved

INVERTER DATA:  Inv.2
|A left outer join has been performed (i.e., keeping all observations of the Inv.2 data)
|The merged dataframe as well as the two dataframes will be saved

INVERTER DATA:  Inv.3
|A left outer join has been performed (i.e., keeping all observations of the Inv.3 data)
|The merged dataframe as well as the two dataframes will be saved

INVERTER DATA:  Inv.4
|A left outer join has been performed (i.e., keeping all observations of the Inv.4 data)
|The merged dataframe as well as the two dataframes will be saved



In [8]:
# TASK: Quick look at the marged dataframe
for inv_name in inv_names:
    print("EXAMPLE: Inverter data {0} [observations: {1} for the period '{2}/{3}']".format(
            inv_name, len(full_inv_data[inv_name]),period_covered[0], period_covered[1]))
    display(full_inv_data[inv_name].sample(5))
    display(full_inv_data[inv_name].info())

EXAMPLE: Inverter data Inv.1 [observations: 149899 for the period '2018-07/2021-06']


,Date/Time,Iac R (A),Iac S (A),Iac T (A),Vac R (V),Vac S (V),Vac T (V),Pac R (kW),Pac S (kW),Pac T (kW),E. totale (kWh),Cc 1 (A),Vcc 1 (V),Allarme,Stato,Inverter temp. (°C),Fac (Hz),Irr. medio (W/mq)
83895,2020-04-28 06:20:00,0,0,0,112,113,113,0,NaN,NaN,3702452.5,0,495,805360192,NaN,65531,NaN,27.0
147436,2021-06-16 13:30:00,125,125,125,118,118,118,45,NaN,NaN,4110207.8,112,406,553701696,NaN,27,NaN,679.0
123994,2021-01-16 11:50:00,559,567,561,118,117,117,200,NaN,NaN,3961775.4,484,437,553701696,NaN,11,NaN,722.0
116157,2020-11-19 17:10:00,0,0,0,112,112,112,0,NaN,NaN,3936819.8,0,27,805360192,NaN,17,NaN,8.0
108261,2020-09-26 10:05:00,0,0,0,111,112,111,0,NaN,NaN,3885667.4,0,462,805360192,NaN,15,NaN,20.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 149899 entries, 0 to 149898
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Date/Time            149899 non-null  datetime64[ns]
 1   Iac R (A)            149899 non-null  int64         
 2   Iac S (A)            149899 non-null  int64         
 3   Iac T (A)            149899 non-null  int64         
 4   Vac R (V)            149899 non-null  int64         
 5   Vac S (V)            149899 non-null  int64         
 6   Vac T (V)            149899 non-null  int64         
 7   Pac R (kW)           149899 non-null  int64         
 8   Pac S (kW)           0 non-null       float64       
 9   Pac T (kW)           0 non-null       float64       
 10  E. totale (kWh)      149899 non-null  float64       
 11  Cc 1 (A)             149899 non-null  int64         
 12  Vcc 1 (V)            149899 non-null  int64         
 13  Allarme       

None

EXAMPLE: Inverter data Inv.2 [observations: 149674 for the period '2018-07/2021-06']


...

None

EXAMPLE: Inverter data Inv.4 [observations: 149281 for the period '2018-07/2021-06']


,Date/Time,Iac R (A),Iac S (A),Iac T (A),Vac R (V),Vac S (V),Vac T (V),Pac R (kW),Pac S (kW),Pac T (kW),E. totale (kWh),Cc 1 (A),Vcc 1 (V),Allarme,Stato,Inverter temp. (°C),Fac (Hz),Irr. medio (W/mq)
37104,2019-06-20 15:20:00,534,537,537,120,120,119,188,NaN,NaN,1385105.1,502,373,553701696,NaN,27,NaN,NaN
118475,2020-12-12 08:15:00,22,19,24,113,113,113,7,NaN,NaN,1930080.3,18,496,553701696,NaN,19,NaN,48.0
54303,2019-10-06 14:00:00,95,87,92,115,116,115,31,NaN,NaN,1518786.4,78,400,553701696,NaN,23,NaN,137.0
5074,2018-09-24 19:00:00,0,0,0,115,116,115,0,NaN,NaN,1153764.1,0,304,805360192,NaN,25,NaN,23.0
138486,2021-04-28 13:40:00,520,528,532,121,120,120,192,NaN,NaN,2042037.4,568,384,553701696,NaN,27,NaN,856.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 149281 entries, 0 to 149280
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Date/Time            149281 non-null  datetime64[ns]
 1   Iac R (A)            149281 non-null  int64         
 2   Iac S (A)            149281 non-null  int64         
 3   Iac T (A)            149281 non-null  int64         
 4   Vac R (V)            149281 non-null  int64         
 5   Vac S (V)            149281 non-null  int64         
 6   Vac T (V)            149281 non-null  int64         
 7   Pac R (kW)           149281 non-null  int64         
 8   Pac S (kW)           0 non-null       float64       
 9   Pac T (kW)           0 non-null       float64       
 10  E. totale (kWh)      149281 non-null  float64       
 11  Cc 1 (A)             149281 non-null  int64         
 12  Vcc 1 (V)            149281 non-null  int64         
 13  Allarme       

None

In [13]:
# TASK: Save dataframes as CSV files
saving_folder_name =  "Imported data"
saving_folder_path = path.join(folder_path, saving_folder_name)

# 0: Create the folders
if not path.exists(saving_folder_path):
    makedirs(saving_folder_path)
    print("A new saving folder has been created")

# 1: Saving merged inverter data
for inv_name in inv_names:
    inv_name_file = inv_name.upper().replace(".", "")
    
    full_inv_data[inv_name].to_csv(path.join(saving_folder_path, "merged_{0}_data.csv".format(inv_name_file)), index=False)
    print("The merged dataframe ({0}) of the {1} has been saved ({2}/{3})"
          .format(len(full_inv_data[inv_name]), inv_name_file, period_covered[0], period_covered[1]))

# 2: Save also all of the irradiance values in case there are some mismatches (i.e., desynchronization) after merging
# MOTIVATION: The discarded irradiance values (due to LeftOuter join) may be useful afterwards
if any(list(perfect_match.values())) == False:   
    irr_dfs.to_csv(path.join(saving_folder_path, "raw_irr_data.csv"), index=False)
    print("\nThe raw irradiance values ({0}) have also been saved (as some desynchronized observations have been found)".format(len(irr_dfs)))

The merged dataframe (149899) of the INV1 has been saved (2018-07/2021-06)
The merged dataframe (149674) of the INV2 has been saved (2018-07/2021-06)
The merged dataframe (148184) of the INV3 has been saved (2018-07/2021-06)
The merged dataframe (149281) of the INV4 has been saved (2018-07/2021-06)

The raw irradiance values (238822) have also been saved (as some desynchronized observations have been found)


In [14]:
df = pd.read_csv(path.join(saving_folder_path, "merged_INV" + "2" + "_data.csv"), 
                 parse_dates=[0], dtype={"Allarme":"string"})
display(df[~df["Irr. medio (W/mq)"].isnull()])
df.info()

,Date/Time,Iac R (A),Iac S (A),Iac T (A),Vac R (V),Vac S (V),Vac T (V),Pac R (kW),Pac S (kW),Pac T (kW),E. totale (kWh),Cc 1 (A),Vcc 1 (V),Allarme,Stato,Inverter temp. (°C),Fac (Hz),Irr. medio (W/mq)
31,2018-07-27 17:00:00,38,36,36,114,114,114,13,NaN,NaN,3047599.9,35,421,553701696,NaN,29,NaN,86.0
32,2018-07-27 17:10:00,61,61,63,114,114,114,21,NaN,NaN,3047603.2,55,422,553701696,NaN,29,NaN,109.0
33,2018-07-27 17:20:00,11,15,15,114,114,114,5,NaN,NaN,3047605.6,11,411,553701696,NaN,27,NaN,49.0
34,2018-07-27 17:30:00,0,0,0,113,113,113,0,NaN,NaN,3047605.7,0,431,805360192,NaN,27,NaN,34.0
35,2018-07-27 17:40:00,0,0,0,113,113,114,0,NaN,NaN,3047605.7,0,416,805360192,NaN,29,NaN,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149668,2021-06-30 21:00:00,0,0,0,109,109,109,0,NaN,NaN,4082158.9,0,86,805360192,NaN,33,NaN,9.0
149669,2021-06-30 21:05:00,0,0,0,108,109,109,0,NaN,NaN,4082158.9,0,22,805360192,NaN,33,NaN,8.0
149670,2021-06-30 21:10:00,0,0,0,109,110,110,0,NaN,NaN,4082158.9,0,20,805360192,NaN,33,NaN,8.0
149671,2021-06-30 21:15:00,0,0,0,109,110,110,0,NaN,NaN,4082158.9,0,20,805360192,NaN,35,NaN,8.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149674 entries, 0 to 149673
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Date/Time            149674 non-null  datetime64[ns]
 1   Iac R (A)            149674 non-null  int64         
 2   Iac S (A)            149674 non-null  int64         
 3   Iac T (A)            149674 non-null  int64         
 4   Vac R (V)            149674 non-null  int64         
 5   Vac S (V)            149674 non-null  int64         
 6   Vac T (V)            149674 non-null  int64         
 7   Pac R (kW)           149674 non-null  int64         
 8   Pac S (kW)           0 non-null       float64       
 9   Pac T (kW)           0 non-null       float64       
 10  E. totale (kWh)      149674 non-null  float64       
 11  Cc 1 (A)             149674 non-null  int64         
 12  Vcc 1 (V)            149674 non-null  int64         
 13  Allarme       